# LETTER-1
##  ("O" positive class)
### Source:
https://archive.ics.uci.edu/ml/datasets/Letter+Recognition
### Goal: 
Identify black-and-white rectangular pixel displays as one of the 26 capital letters in the English alphabet

# IMPORTS

In [1]:
import numpy as np
import pandas as pd
import requests, zipfile, io

from sklearn.metrics import accuracy_score, f1_score, auc, roc_auc_score, roc_curve, make_scorer
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

#plots
import seaborn as sns
import matplotlib.pyplot as plt

# Algo 1: LOGISTIC REGRESSION

In [2]:
def log_reg_classifier(X_train, Y_train, X_test, Y_test):
    # define hyperparameters to search through & error metrics 
    C_list = [10**-8,10**-7,10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,10**0,10**1,10**2,10**3,10**4]
    penalty_list = ['l2', 'l1', 'none']
    solver = ['saga'] #this solver works for no penalty, l1 and l2 penalties
    scoring = {'Accuracy': make_scorer(accuracy_score), 'f1': 'f1', 'AUC': 'roc_auc'}
    classifier = LogisticRegression()
        
    # gridsearch 
    lr =  GridSearchCV(classifier, {'C':C_list, 'penalty':penalty_list,'solver':solver},
                                   scoring=scoring,refit=False, cv=5, n_jobs=-1, 
                                   return_train_score=True, verbose=2 ) 
        
    lr.fit(X_train,Y_train)
    results = pd.DataFrame(lr.cv_results_)
        
    # Get each parameter settings that gives best accuracy, F1, and AUC on validation set
    best_AUC = results[results['rank_test_AUC']==1]['params'].values[0]
    best_accuracy = results[results['rank_test_Accuracy']  ==1]['params'].values[0]
    best_F1 = results[results['rank_test_f1']  ==1]['params'].values[0]
        
    # Train 3 models using the 5000 samples and each of the 3 best parameter settings (one model per metric)
    clf_AUC = LogisticRegression(C=best_AUC['C'], solver = best_AUC['solver'],
                             penalty=best_AUC['penalty']).fit(X_train, Y_train)

    clf_accuracy = LogisticRegression(C=best_accuracy['C'], solver = best_accuracy['solver'],
                             penalty=best_accuracy['penalty']).fit(X_train, Y_train)

    clf_F1 = LogisticRegression(C=best_F1['C'], solver = best_F1['solver'],
                                penalty=best_F1['penalty']).fit(X_train, Y_train)
        
    # For average training performance
    train_accuracy = clf_accuracy.score(X_train, Y_train)  # Accuracy  
    train_F1 = f1_score(Y_train, clf_F1.predict(X_train))   # F1
    fpr, tpr, threshold = roc_curve(Y_train, clf_AUC.predict_proba(X_train)[:,1])
    train_auc = auc(fpr, tpr) # AUC
        
    # Find and store accuracy, F1, and AUC of the 3 models from previous line on test set
    test_accuracy = clf_accuracy.score(X_test, Y_test)  # Accuracy
    
    test_F1 = f1_score(Y_test, clf_F1.predict(X_test))   # F1
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, clf_AUC.predict_proba(X_test)[:,1])
    test_auc = auc(false_positive_rate, true_positive_rate) # AUC

    return train_accuracy, train_F1, train_auc, test_accuracy, test_F1, test_auc


# Algo 2: RANDOM FOREST

In [3]:
def rf_classifier(X_train,Y_train,X_test,Y_test):
    # define hyperparameters to search through & error metrics  
    n_estimators = [1024]
    min_samples_split = [1,2,4,6,8,12,16,20]
    scoring = {'Accuracy': make_scorer(accuracy_score), 'f1': 'f1', 'AUC': 'roc_auc'}
    classifier = RandomForestClassifier()

    rf =  GridSearchCV(classifier, {'n_estimators':n_estimators, 'min_samples_split':min_samples_split},
                                   scoring=scoring,refit=False, cv=5, n_jobs=-1, 
                                   return_train_score=True, verbose=2) 

    rf.fit(X_train,Y_train)
    results = pd.DataFrame(rf.cv_results_)
        
    # Get each parameter settings that gives best accuracy, F1, and AUC on validation set
    best_AUC = results[results['rank_test_AUC']==1]['params'].values[0]
    best_accuracy = results[results['rank_test_Accuracy']  ==1]['params'].values[0]
    best_F1 = results[results['rank_test_f1']  ==1]['params'].values[0]

    # Train 3 models using the 5000 samples and each of the 3 best parameter settings (one model per metric)
    clf_AUC = RandomForestClassifier(n_estimators=best_AUC['n_estimators'], 
                              min_samples_split=best_AUC['min_samples_split']).fit(X_train, Y_train)

    clf_accuracy = RandomForestClassifier(n_estimators=best_accuracy['n_estimators'], 
                              min_samples_split=best_accuracy['min_samples_split']).fit(X_train, Y_train)

    clf_F1 = RandomForestClassifier(n_estimators=best_F1['n_estimators'], 
                                  min_samples_split=best_F1['min_samples_split']).fit(X_train, Y_train)

    # For average training performance
    train_accuracy = clf_accuracy.score(X_train, Y_train)  # Accuracy  
    train_F1 = f1_score(Y_train, clf_F1.predict(X_train))   # F1
    fpr, tpr, threshold = roc_curve(Y_train, clf_AUC.predict_proba(X_train)[:,1])
    train_auc = auc(fpr, tpr) # AUC
        
    # Find and store accuracy, F1, and AUC of the 3 models from previous line on test set
    test_accuracy = clf_accuracy.score(X_test, Y_test)  # Accuracy
    
    test_F1 = f1_score(Y_test, clf_F1.predict(X_test))   # F1
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, clf_AUC.predict_proba(X_test)[:,1])
    test_auc = auc(false_positive_rate, true_positive_rate) # AUC

    return train_accuracy, train_F1, train_auc, test_accuracy, test_F1, test_auc
  

# Algo 3: KNN

In [4]:
def knn_classifier(X_train,Y_train,X_test,Y_test):
    # define hyperparameters to search through & error metrics  
    n_neighbors = np.linspace(1, 105, num=27, dtype=int)
    weights = ['uniform', 'distance']
    p = [1,2]
    scoring = {'Accuracy': make_scorer(accuracy_score), 'f1': 'f1', 'AUC': 'roc_auc'}
    classifier = KNeighborsClassifier()

    rf =  GridSearchCV(classifier, {'n_neighbors':n_neighbors, 'weights': weights, 'p':p},
                                   scoring=scoring,refit=False, cv=5, n_jobs=-1, 
                                   return_train_score=True, verbose=2) 

    rf.fit(X_train,Y_train)
    results = pd.DataFrame(rf.cv_results_)
        
    # Get each parameter settings that gives best accuracy, F1, and AUC on validation set
    best_AUC = results[results['rank_test_AUC']==1]['params'].values[0]
    best_accuracy = results[results['rank_test_Accuracy']  ==1]['params'].values[0]
    best_F1 = results[results['rank_test_f1']  ==1]['params'].values[0]

    # Train 3 models using the 5000 samples and each of the 3 best parameter settings (one model per metric)
    clf_AUC = KNeighborsClassifier(n_neighbors=best_AUC['n_neighbors'], 
                             weights=best_AUC['weights'], p = best_AUC['p']).fit(X_train, Y_train)

    clf_accuracy = KNeighborsClassifier(n_neighbors=best_accuracy['n_neighbors'], 
                              weights=best_accuracy['weights']).fit(X_train, Y_train)

    clf_F1 = KNeighborsClassifier(n_neighbors=best_F1['n_neighbors'], 
                                  weights=best_F1['weights'], p = best_F1['p']).fit(X_train, Y_train)

    # For average training performance
    train_accuracy = clf_accuracy.score(X_train, Y_train)  # Accuracy  
    train_F1 = f1_score(Y_train, clf_F1.predict(X_train))   # F1
    fpr, tpr, threshold = roc_curve(Y_train, clf_AUC.predict_proba(X_train)[:,1])
    train_auc = auc(fpr, tpr) # AUC
        
    # Find and store accuracy, F1, and AUC of the 3 models from previous line on test set
    test_accuracy = clf_accuracy.score(X_test, Y_test)  # Accuracy
    
    test_F1 = f1_score(Y_test, clf_F1.predict(X_test))   # F1
    
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, clf_AUC.predict_proba(X_test)[:,1])
    test_auc = auc(false_positive_rate, true_positive_rate) # AUC

    return train_accuracy, train_F1, train_auc, test_accuracy, test_F1, test_auc  

# DATA

In [5]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data', header=None)
print(data.shape)
data.head()

(20000, 17)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [6]:
#checking if any observations have missing data
rows_to_drop = data[data.isnull().any(axis=1)] 
print(rows_to_drop)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
Index: []


In [7]:
# get inputs (X) & targets (y) 
X = data.loc[: , 1:16] #feature inputs: taking colums "1" to "16"
labels = data[0]


#transform

#determining +/- classes, O = positive class, everything else = negative class
Y=[]
for i in labels:
    if i in ('O'):
        Y += [1]
    else:
        Y += [-1]
        
data['y'] = Y

In [8]:
# %POZ
print(data['y'].value_counts())

percent_positive = 753/20000
print(percent_positive)

-1    19247
 1      753
Name: y, dtype: int64
0.03765


### 5 Trials across the three algos 

In [9]:
log_reg_data = []
rf_data = []
knn_data = []

for i in range(5):
    #split data for training and testing
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, train_size=5000)

        
    #tansform data for Logistic Regression and KNN
    X_train_t = StandardScaler().fit_transform(X_train)    
    X_test_t = StandardScaler().fit_transform(X_test)
    
    log_reg_data += log_reg_classifier(X_train_t, Y_train, X_test_t, Y_test)
    rf_data += rf_classifier(X_train,Y_train, X_test, Y_test)
    knn_data += knn_classifier(X_train_t,Y_train, X_test_t, Y_test)
    
    

Fitting 5 folds for each of 39 candidates, totalling 195 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 195 out of 195 | elapsed:   10.0s finished
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.3min finished


Fitting 5 folds for each of 108 candidates, totalling 540 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 195 out of 195 | elapsed:    8.9s finished
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/annaleaohalloran/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.3min finished


Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 11.4min finished


In [15]:
#Logistic Regression Results
df_log_reg = pd.DataFrame(columns=('Train ACC', 'Train F1','Train AUC', 'Test ACC', 'Test F1', 'Test AUC'))
df_log_reg.loc['LR Letter 1 Trial 1'] = log_reg_data[0:6]
df_log_reg.loc['LR Letter 1 Trial 2'] = log_reg_data[6:12]
df_log_reg.loc['LR Letter 1 Trial 3'] = log_reg_data[12:18]
df_log_reg.loc['LR Letter 1 Trial 4'] = log_reg_data[18:24]
df_log_reg.loc['LR Letter 1 Trial 5'] = log_reg_data[24:32]

#Random Forest Results
df_rf = pd.DataFrame(columns=('Train ACC', 'Train F1','Train AUC', 'Test ACC', 'Test F1', 'Test AUC'))
df_rf.loc['RF Letter 1 Trial 1'] = rf_data[0:6]
df_rf.loc['RF Letter 1 Trial 2'] = rf_data[6:12]
df_rf.loc['RF Letter 1 Trial 3'] = rf_data[12:18]
df_rf.loc['RF Letter 1 Trial 4'] = rf_data[18:24]
df_rf.loc['RF Letter 1 Trial 5'] = rf_data[24:32]


#KNN Results
df_knn = pd.DataFrame(columns=('Train ACC', 'Train F1','Train AUC', 'Test ACC', 'Test F1', 'Test AUC'))
df_knn.loc['KNN Letter 1 Trial 1'] = knn_data[0:6]
df_knn.loc['KNN Letter 1 Trial 2'] = knn_data[6:12]
df_knn.loc['KNN Letter 1 Trial 3'] = knn_data[12:18]
df_knn.loc['KNN Letter 1 Trial 4'] = knn_data[18:24]
df_knn.loc['KNN Letter 1 Trial 5'] = knn_data[24:32]

print('Logistic Regression Results')
display(df_log_reg)
print(df_log_reg.mean())

print('\n\nRandom Forest Results')
display(df_rf)
print(df_rf.mean())

print('\n\nKNN Results')
display(df_knn)
print(df_knn.mean())


Logistic Regression Results


,Train ACC,Train F1,Train AUC,Test ACC,Test F1,Test AUC
LR Letter 1 Trial 1,0.9618,0.0,0.875447,0.962533,0.0,0.866262
LR Letter 1 Trial 2,0.9618,0.0,0.858784,0.962533,0.0,0.846642
LR Letter 1 Trial 3,0.9632,0.0,0.864379,0.962067,0.0,0.858188
LR Letter 1 Trial 4,0.9628,0.0,0.866015,0.962200,0.0,0.853746
LR Letter 1 Trial 5,0.9624,0.0,0.863934,0.962333,0.0,0.844450


Train ACC    0.962400
Train F1     0.000000
Train AUC    0.865712
Test ACC     0.962333
Test F1      0.000000
Test AUC     0.853858
dtype: float64


Random Forest Results


,Train ACC,Train F1,Train AUC,Test ACC,Test F1,Test AUC
RF Letter 1 Trial 1,1.0,1.000000,1.0,0.988067,0.809573,0.997058
RF Letter 1 Trial 2,1.0,1.000000,1.0,0.988467,0.830491,0.997183
RF Letter 1 Trial 3,1.0,0.994536,1.0,0.986867,0.797101,0.997514
RF Letter 1 Trial 4,1.0,1.000000,1.0,0.986867,0.792492,0.996502
RF Letter 1 Trial 5,1.0,1.000000,1.0,0.987133,0.795812,0.996441


Train ACC    1.000000
Train F1     0.998907
Train AUC    1.000000
Test ACC     0.987480
Test F1      0.805094
Test AUC     0.996940
dtype: float64


KNN Results


,Train ACC,Train F1,Train AUC,Test ACC,Test F1,Test AUC
KNN Letter 1 Trial 1,1.000,1.000000,1.0,0.991400,0.887728,0.995656
KNN Letter 1 Trial 2,1.000,1.000000,1.0,0.990200,0.855184,0.993553
KNN Letter 1 Trial 3,0.996,0.945652,1.0,0.988733,0.849778,0.996428
KNN Letter 1 Trial 4,1.000,1.000000,1.0,0.991467,0.900804,0.996031
KNN Letter 1 Trial 5,1.000,1.000000,1.0,0.990400,0.871429,0.992319


Train ACC    0.999200
Train F1     0.989130
Train AUC    1.000000
Test ACC     0.990440
Test F1      0.872985
Test AUC     0.994797
dtype: float64


In [16]:
# need to save the data for ttests
df_log_reg.to_csv("lr_letter1.csv")
df_rf.to_csv("rf_letter1.csv")
df_knn.to_csv("knn_letter1.csv")